In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
df = pd.read_csv("../Test/vehicles.csv")

# Data preprocessing

In [3]:
# Drop columns not needed and NaN 
df_columns = df.drop(columns=['url','region','region_url','VIN','size','paint_color','image_url','description','county','state','lat','long','posting_date'])
df_columns_drop = df_columns.dropna()

In [4]:
# Data conversion
df_columns_drop['id'] = df_columns_drop['id'].astype(str)

df_columns_drop['year'] = df_columns_drop['year'].astype('int')
df_columns_drop['year'] = pd.to_datetime(df_columns_drop['year'].astype(str))

df_columns_drop['cylinders'] = df_columns_drop['cylinders'].str.replace(' cylinders','')
df_columns_drop['cylinders'] = df_columns_drop['cylinders'].str.replace('other','0')
df_columns_drop['cylinders'] = df_columns_drop['cylinders'].astype('int')

C:\Users\happi\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\happi\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\happi\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [5]:
# Remove outliers in price
df_price = df_columns_drop[df_columns_drop['price'] <= 100000]

In [6]:
# In addition, keep data between 2001 and 2020
df_date = df_price[(df_price['year'] >= dt.datetime(2001,1,1)) & (df_price['year'] < dt.datetime(2021,1,1))]

In [7]:
# Remove outliers in odometer
df_od = df_date[df_date['odometer'] <= 200000]

In [8]:
# Select 'clean' for title status, 'gas' for fuel, and 'automatic' for transmission
df_std = df_od[(df_od['title_status'] == 'clean') & (df_od['fuel'] == 'gas') & (df_od['transmission'] == 'automatic')]

In [9]:
# Specify outliers in each year
outlier_2001 = df_std.index[(df_std['year'] == dt.datetime(2001,1,1)) & (df_std['price'] > 40000)]
outlier_2002 = df_std.index[(df_std['year'] == dt.datetime(2002,1,1)) & (df_std['price'] > 40000)]

In [10]:
# Remove specified outliers
df_std_drop = df_std.drop(outlier_2001)
df_std_drop = df_std_drop.drop(outlier_2002)

In [12]:
# Select columns for analysis
df_test = df_std_drop.loc[:,['price','year','cylinders','odometer','drive','condition']]

In [21]:
# Create our features and target
X = df_test.copy().drop('price', axis=1)
X = pd.get_dummies(X, columns=['year','drive','condition'])
y = df_test['price']

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [22]:
# Create training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1)

In [30]:
# Create model
regr_1 = DecisionTreeRegressor(max_depth=5)
regr_2 = DecisionTreeRegressor(max_depth=10)
regr_3 = DecisionTreeRegressor(max_depth=15)

In [31]:
# Fit model
regr_1.fit(X_train, y_train)
regr_2.fit(X_train, y_train)
regr_3.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=15)

In [25]:
# Make prediction 1 
y_pred_1 = regr_1.predict(X_test)

In [26]:
# Make prediction 2
y_pred_2 = regr_2.predict(X_test)

In [32]:
# Make prediction 3
y_pred_3 = regr_3.predict(X_test)

In [33]:
# Result of prediction
data = {'price': y_test,
       'prediction 1': y_pred_1,
       'prediction 2': y_pred_2,
       'prediction 3': y_pred_3}
df_pred = pd.DataFrame(data)

In [35]:
df_pred

,price,prediction 1,prediction 2,prediction 3
15848,9995,12624.927677,9936.761079,11483.583333
92420,3995,8609.802217,7036.790550,5464.692308
250543,7495,8609.802217,9582.125275,8091.240000
358288,10743,8609.802217,7036.790550,6789.116418
179232,17600,23098.326180,22275.525915,19367.952381
...,...,...,...,...
62514,59950,26255.227355,28875.434146,31390.942308
158172,15900,12624.927677,9936.761079,14630.115207
4774,10500,7339.107227,9414.132653,10539.585938
39820,14995,8609.802217,7036.790550,11041.906250
